In [ ]:
# Libraries

# Standard library imports
import os
import logging

# Selenium imports
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Variables

# Directory files should be downloaded to 
download_dir = os.path.abspath("./Outputs/Downloads")

# Directory if using a custom chrome profile
profile_dir = os.path.expanduser("~") + r"\AppData\Local\Google\Chrome\User Data\Developer"

# Other options
headless = False
notifications = False
media_stream = False
geolocation = False
local_discovery = False

In [ ]:
# Set up logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(), logging.FileHandler("sandbox.log")]
)

In [ ]:
# Create a Chrome WebDriver with Options
def create_driver(headless=False, profile_dir=None, download_dir=None):
    chrome_options = Options()

    # Optional: Run headless for environments without GUI
    if headless:
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--window-size=1920x1080")  # Set default window size for headless
    
    # Optional: Load custom profile (to retain session, cookies, etc.)
    if profile_dir:
        chrome_options.add_argument(f"user-data-dir={profile_dir}")
    
    # Stability options
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--disable-cache")
    chrome_options.add_argument("--disable-extensions")

    # Download preferences
    if download_dir:
        # Create download directory if it doesn't exist
        os.makedirs(download_dir, exist_ok=True)
        logging.info(f"Download directory set to: {download_dir}")
        
        prefs = {
            "download.default_directory": download_dir, # Set default download directory
            "download.prompt_for_download": False, # Disable download prompt
            "download.directory_upgrade": True, # Auto-update download directory
            "plugins.always_open_pdf_externally": True, # Download PDFs instead of opening them
            "profile.default_content_setting_values": {
                "notifications": 2,  # Block notifications
                "media_stream": 2,  # Block camera/microphone
                "geolocation": 2,  # Block location
                "local_discovery": 2  # Block local network discovery
            }
        }
        chrome_options.add_experimental_option("prefs", prefs)
    
    # Create and return the Chrome WebDriver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.implicitly_wait(10)  # Set default implicit wait
    logging.info("Chrome WebDriver created successfully.")
    return driver


# Single Driver Example

Single driver, good for linear scripts and testing

In [ ]:
logging.info("Initializing WebDriver...")
driver = create_driver(
    headless = headless,
    profile_dir = profile_dir,
    download_dir = download_dir
    )
logging.info("WebDriver initialized.")

# Concurrency Example

Example of concurrency using multiple drivers and a worker pool

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def worker_search(worker_id, search_query, headless=False):
    """
    Worker function that creates a driver, performs a Google search, and closes.
    
    Args:
        worker_id: Unique identifier for this worker
        search_query: Query to search on Google
        headless: Whether to run in headless mode
        
    Returns:
        dict: Results of the operation
    """
    worker_driver = None
    try:
        logging.info(f"Worker {worker_id}: Starting...")
        
        # Create a new driver for this worker
        worker_driver = create_driver(headless=headless)
        
        # Navigate to Google
        logging.info(f"Worker {worker_id}: Navigating to Google...")
        worker_driver.get("https://www.google.com")
        
        # Wait for and find the search box
        search_box = WebDriverWait(worker_driver, 10).until(
            EC.presence_of_element_located((By.NAME, "q"))
        )
        
        # Perform search
        logging.info(f"Worker {worker_id}: Searching for '{search_query}'...")
        search_box.send_keys(search_query)
        search_box.submit()
        
        # Wait for results to load
        WebDriverWait(worker_driver, 10).until(
            EC.presence_of_element_located((By.ID, "search"))
        )
        
        # Get the page title
        page_title = worker_driver.title
        logging.info(f"Worker {worker_id}: Search completed. Page title: {page_title}")
        
        # Simulate some work
        time.sleep(2)
        
        return {
            "worker_id": worker_id,
            "query": search_query,
            "title": page_title,
            "success": True
        }
        
    except Exception as e:
        logging.error(f"Worker {worker_id}: Error occurred - {str(e)}")
        return {
            "worker_id": worker_id,
            "query": search_query,
            "success": False,
            "error": str(e)
        }
        
    finally:
        # Always close the driver
        if worker_driver:
            logging.info(f"Worker {worker_id}: Closing driver...")
            worker_driver.quit()


# Configuration
num_workers = 3
search_queries = [
    "Python Selenium tutorial",
    "Web scraping best practices",
    "Chrome WebDriver automation",
    "Selenium Grid setup",
    "Cross-browser testing with Selenium",
    "10 Reasons why Firefox is better than Chrome",
    "Install UBlock Origin Today"
]

# Execute searches concurrently
logging.info(f"Starting {num_workers} concurrent workers...")
results = []

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    # Submit all tasks
    futures = {
        executor.submit(worker_search, i+1, search_queries[i], headless): i 
        for i in range(num_workers)
    }
    
    # Collect results as they complete
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        
        if result["success"]:
            logging.info(f"Worker {result['worker_id']} completed successfully")
        else:
            logging.error(f"Worker {result['worker_id']} failed: {result.get('error', 'Unknown error')}")

# Summary
logging.info("\n" + "="*50)
logging.info("CONCURRENCY EXAMPLE COMPLETE")
logging.info("="*50)
successful = sum(1 for r in results if r["success"])
logging.info(f"Total workers: {num_workers}")
logging.info(f"Successful: {successful}")
logging.info(f"Failed: {num_workers - successful}")
logging.info("="*50)


# Microsoft Login Example

The `microsoft_login` utility automates Microsoft authentication:
1. Enter username and click "Next"
2. Enter password and click "Sign in"
3. Handle "Stay signed in?" prompt

It automatically loads credentials from the `.env` file.

In [ ]:
# Import the Microsoft login utility
from Utilities.microsoft_login import microsoft_login, wait_for_login_redirect

# Perform login (uses credentials from .env file)
try:
    success = microsoft_login(driver, stay_signed_in=False, timeout=20)
    
    if success:
        logging.info("Microsoft login successful!")
        
        # Optional: Wait for redirect to specific page
        # wait_for_login_redirect(driver, expected_url="portal.office.com")
    else:
        logging.error("Microsoft login failed!")
        
except Exception as e:
    logging.error(f"Login error: {e}")